In [ ]:
#!wget https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt
#filename = 't8.shakespeare.txt'
!wget https://norvig.com/big.txt
filename = 'big.txt'

--2023-05-01 21:27:23--  https://norvig.com/big.txt
Resolving norvig.com (norvig.com)... 158.106.138.13
Connecting to norvig.com (norvig.com)|158.106.138.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6488666 (6.2M) [text/plain]
Saving to: ‘big.txt’

big.txt             100%[===================>]   6.19M  11.3MB/s    in 0.5s    

2023-05-01 21:27:24 (11.3 MB/s) - ‘big.txt’ saved [6488666/6488666]



In [ ]:
with open(filename, 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
print("Length of text: ", len(text))

Length of text:  6488666


In [ ]:
print(text[:1000])

The Project Gutenberg EBook of The Adventures of Sherlock Holmes
by Sir Arthur Conan Doyle
(#15 in our series by Sir Arthur Conan Doyle)

Copyright laws are changing all over the world. Be sure to check the
copyright laws for your country before downloading or redistributing
this or any other Project Gutenberg eBook.

This header should be the first thing seen when viewing this Project
Gutenberg file.  Please do not remove it.  Do not change or edit the
header without written permission.

Please read the "legal small print," and other information about the
eBook and Project Gutenberg at the bottom of this file.  Included is
important information about your specific rights and restrictions in
how the file may be used.  You can also find out about how to make a
donation to Project Gutenberg, and how to get involved.


**Welcome To The World of Free Plain Vanilla Electronic Texts**

**eBooks Readable By Both Humans and By Computers, Since 1971**

*****These eBooks Were Prepared By Thousan

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print("size: ", vocab_size)

	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_abcdefghijklmnopqrstuvwxyz|~
size:  93


In [ ]:
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("I love you"))
print(decode(encode("I love you")))

[43, 2, 76, 79, 86, 69, 2, 89, 79, 85]
I love you


In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([6488666]) torch.int64
tensor([54, 72, 69,  2, 50, 82, 79, 74, 69, 67, 84,  2, 41, 85, 84, 69, 78, 66,
        69, 82, 71,  2, 39, 36, 79, 79, 75,  2, 79, 70,  2, 54, 72, 69,  2, 35,
        68, 86, 69, 78, 84, 85, 82, 69, 83,  2, 79, 70,  2, 53, 72, 69, 82, 76,
        79, 67, 75,  2, 42, 79, 76, 77, 69, 83,  1, 66, 89,  2, 53, 73, 82,  2,
        35, 82, 84, 72, 85, 82,  2, 37, 79, 78, 65, 78,  2, 38, 79, 89, 76, 69,
         1, 10,  5, 19, 23,  2, 73, 78,  2, 79, 85, 82,  2, 83, 69, 82, 73, 69,
        83,  2, 66, 89,  2, 53, 73, 82,  2, 35, 82, 84, 72, 85, 82,  2, 37, 79,
        78, 65, 78,  2, 38, 79, 89, 76, 69, 11,  1,  1, 37, 79, 80, 89, 82, 73,
        71, 72, 84,  2, 76, 65, 87, 83,  2, 65, 82, 69,  2, 67, 72, 65, 78, 71,
        73, 78, 71,  2, 65, 76, 76,  2, 79, 86, 69, 82,  2, 84, 72, 69,  2, 87,
        79, 82, 76, 68, 16,  2, 36, 69,  2, 83, 85, 82, 69,  2, 84, 79,  2, 67,
        72, 69, 67, 75,  2, 84, 72, 69,  1, 67, 79, 80, 89, 82, 73, 71, 72, 84,
      

In [ ]:
# Setting up train and validation sets
split = int(0.9 * len(data))
train_data = data[:split]
val_data = data[split:]

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([54, 72, 69,  2, 50, 82, 79, 74, 69])

In [ ]:
# illustrating training on a block
# we take a sequence of chars (len 1 to block_size), and see what appears next
x = train_data[:block_size]
y = train_data[1:1+block_size]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"When input is {context} then the target is {target}")

When input is tensor([54]) then the target is 72
When input is tensor([54, 72]) then the target is 69
When input is tensor([54, 72, 69]) then the target is 2
When input is tensor([54, 72, 69,  2]) then the target is 50
When input is tensor([54, 72, 69,  2, 50]) then the target is 82
When input is tensor([54, 72, 69,  2, 50, 82]) then the target is 79
When input is tensor([54, 72, 69,  2, 50, 82, 79]) then the target is 74
When input is tensor([54, 72, 69,  2, 50, 82, 79, 74]) then the target is 69


In [ ]:
torch.manual_seed(989)
batch_size = 4 # how many independent sequences to process in parallel
block_size = 8

def get_batch(split):
  # generate a small batch of data of inputs x and targets y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+1+block_size] for i in ix])
  return x, y

# here's what gets fed into the neural network
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('====')

# illustrating the training again
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"When input is {context} then the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[68, 73, 86, 69, 82, 83, 73, 84],
        [82, 69, 83, 69, 77, 66, 76, 73],
        [68,  2, 87, 72, 65, 84,  2, 89],
        [79, 79, 84, 83, 84, 69, 80, 83]])
targets:
torch.Size([4, 8])
tensor([[73, 86, 69, 82, 83, 73, 84, 89],
        [69, 83, 69, 77, 66, 76, 73, 78],
        [ 2, 87, 72, 65, 84,  2, 89, 79],
        [79, 84, 83, 84, 69, 80, 83,  2]])
====
When input is tensor([68]) then the target is 73
When input is tensor([68, 73]) then the target is 86
When input is tensor([68, 73, 86]) then the target is 69
When input is tensor([68, 73, 86, 69]) then the target is 82
When input is tensor([68, 73, 86, 69, 82]) then the target is 83
When input is tensor([68, 73, 86, 69, 82, 83]) then the target is 73
When input is tensor([68, 73, 86, 69, 82, 83, 73]) then the target is 84
When input is tensor([68, 73, 86, 69, 82, 83, 73, 84]) then the target is 89
When input is tensor([82]) then the target is 69
When input is tensor([82, 69]) then the target is

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(989)

class BigramLanguageModel(nn.Module):
  def __init__(me, vocab_size):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    me.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(me, idx, targets=None):
    # idx and targets are both (B,T) tensor of integers
    logits = me.token_embedding_table(idx) # (B,T,C)

    if targets is None:
      loss = None
    else:
      # reshape the matrix to comply to pytorch's spec
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss
  
  def generate(me, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      # get the prediction
      logits, loss = me(idx)
      # focus only on the last time step
      logits = logits[:, -1, :] # becomes (B,C)
      probs = F.softmax(logits, dim=-1) # (B,C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
      idx = torch.cat((idx, idx_next), dim=1) # (B,T+1)
    return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# try generating, which will output garbage since we haven't trained
# but at least we can see that the plumbings work
gen_result = m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)
print(decode(gen_result[0].tolist()))

torch.Size([32, 93])
tensor(4.9968, grad_fn=<NllLossBackward0>)
	jgx5-jl14C1;Mr;v~(feQ<Vm=&=iRcP#I+M%zCeyld8Q0+9]k2:!k2.R
%,gE]oz(	5sn3cbUSB>I4yZ,eMG()ounv/k~kOK).h(


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1.e-3)

In [ ]:
batch_size = 32

# train the model
for steps in range(10000):
  #sample a batch of data
  xb, yb = get_batch('train')

  #evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.497157573699951


In [ ]:
# retry the generation with trained model
gen_result = m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)
print(decode(gen_result[0].tolist()))

	).
an d d ak th

on or ty tar heran cuthov#Th, n aremandy MarsqurenKr e. ban
torunes,Ohomot
r an thett bymi'ssofof I tins omap co:
cett wng avewe ueaxcedgend Did maler, oly te akwe mon
"

bo Hexpweait erit to de
 abn thevice, s g winov..  st ke, a Prous d h cowh sestok, his ad ca. r. ine t thing indind If audlisht d w$"t d Bodjone thin an ncind inamusase ofans ofiRUn f whem brgre
Thatiteplof
Kull 


In [ ]:
# Mathematical trick in Self Attention

# example, to make tokens get info from previous ones (but not future ones)
torch.manual_seed(444)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
print(x.shape)

# one lossy way to do that is to average all previous tokens plus itself
# x[b,t] = mean_{i<=t} x[b,i]
# version 1: brute force with for-loops
xbow = torch.zeros((B,T,C)) # x-bag-of-words
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1] # (t, C)
    xbow[b,t] = torch.mean(xprev, 0)

torch.Size([4, 8, 2])


In [ ]:
print("original\n", x)

original
 tensor([[[ 2.0291,  0.9115],
         [ 0.2897, -1.8648],
         [ 0.2171, -1.0556],
         [ 1.8147,  1.5656],
         [ 0.2540,  0.0917],
         [-0.7039,  1.5540],
         [-0.9957,  0.1390],
         [-0.1338, -0.1376]],

        [[ 1.1425, -0.4524],
         [-0.7752, -1.6074],
         [-0.3859, -0.0484],
         [ 1.2081,  0.0876],
         [-0.5788,  1.1616],
         [ 0.7847, -0.2466],
         [ 0.9276, -1.5267],
         [ 0.7774, -0.0511]],

        [[ 0.3798,  2.5237],
         [ 0.8944, -0.4245],
         [-0.0309,  0.6419],
         [ 0.5361, -0.9341],
         [ 1.1554,  1.1066],
         [ 0.0873, -0.1423],
         [-2.4982, -1.9596],
         [ 0.4957,  2.2735]],

        [[ 0.2789, -1.0112],
         [-1.3280,  0.7159],
         [ 1.9788,  0.6944],
         [ 1.3840,  0.5109],
         [ 1.2745,  1.0589],
         [-0.5768, -0.0208],
         [-1.0459,  1.3953],
         [-0.9601,  0.6664]]])


In [ ]:
print("x-bag-of-words\n", xbow[3])

x-bag-of-words
 tensor([[ 0.2789, -1.0112],
        [-0.5246, -0.1477],
        [ 0.3099,  0.1330],
        [ 0.5784,  0.2275],
        [ 0.7176,  0.3938],
        [ 0.5019,  0.3247],
        [ 0.2808,  0.4776],
        [ 0.1257,  0.5012]])


In [ ]:
# the above works, but uses slow for-loops
# version 2: improve with matrix multiplication trick
a = torch.tril(torch.ones(3, 3)) # lower triangle section have ones
print("* tril a =\n", a)
a = a / torch.sum(a, 1, keepdim=True)
print("* tril a normalized =\n", a)
b = torch.randint(1, 10, (3, 2)).float()
print("* rand int b =\n", b)
c = a @ b
print("* moving avg of b -> c =\n", c)

* tril a =
 tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
* tril a normalized =
 tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
* rand int b =
 tensor([[7., 7.],
        [8., 2.],
        [4., 9.]])
* moving avg of b -> c =
 tensor([[7.0000, 7.0000],
        [7.5000, 4.5000],
        [6.3333, 6.0000]])


In [ ]:
# now do the moving average on the weights
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B,T,T) @ (B,T,C) -> (B,T,C)
assert(torch.allclose(xbow, xbow2)) # equals to xbow, with faster calculation
xbow2[3]

tensor([[ 0.2789, -1.0112],
        [-0.5246, -0.1477],
        [ 0.3099,  0.1330],
        [ 0.5784,  0.2275],
        [ 0.7176,  0.3938],
        [ 0.5019,  0.3247],
        [ 0.2808,  0.4776],
        [ 0.1257,  0.5012]])

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
print(wei) # same moving avg matrix multiplier, but created with softmax
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


True

In [ ]:
# version 4: Self Attention!
torch.manual_seed(123)
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B,T,16)
q = query(x) # (B,T,16)
wei = q @ k.transpose(-2, -1) # (B,T,16) @(B,16,T) -> (B,T,T)

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

# out = wei @ x
v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2461, 0.7539, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1788, 0.0721, 0.7490, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0198, 0.0119, 0.9663, 0.0021, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0868, 0.0224, 0.6254, 0.0866, 0.1788, 0.0000, 0.0000, 0.0000],
        [0.0674, 0.2265, 0.0070, 0.2752, 0.2119, 0.2120, 0.0000, 0.0000],
        [0.0844, 0.0189, 0.6506, 0.0281, 0.0320, 0.1514, 0.0346, 0.0000],
        [0.0332, 0.0784, 0.1998, 0.1428, 0.1057, 0.0981, 0.3181, 0.0240]],
       grad_fn=<SelectBackward0>)

Notes:
* Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at eadh other and aggregating information with a wieghted sum from all nodes that point to them, with data-dependent weights.
* There is no notion of space. Attention simply acts over a set of vectors. This is why we need to  positionally encode tokens.
* Each example across batch dimension is of course processed completely independently and never "talk" to each other
* In an "incoder" attention block just delete the single line that does making with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modelling.
* "self attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
* "Scaled" attention additional deivides wei by 1/sqrt(head_size). This makes it so when input Q, K are unit variance, we will be unite variance too and Softmax will stay diffuse and not saturate too much.

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1)
print(f"{wei.shape}\n{wei[0]}\n{k.var()}, {q.var()}, {wei.var()}")
wei = q @ k.transpose(-2, -1) * head_size**-0.5 # scaled to 1/sqrt(head_size)
print(f"{wei.shape}\n{wei[0]}\n{k.var()}, {q.var()}, {wei.var()}")

torch.Size([4, 8, 8])
tensor([[ 0.8493,  5.2974, -2.2423, -0.4562, -0.1847,  6.5117, -1.2332,  7.0318],
        [ 0.8963,  0.1012,  2.8573,  1.0340, -1.8438, -3.6918,  1.7688, -2.6806],
        [ 2.7227,  2.5355, -0.3369,  0.9038, -1.0929, -3.0553,  0.8732,  1.9568],
        [ 2.7093,  3.1052,  3.2472,  4.1662, -0.2696, -1.7073,  0.9370,  0.1977],
        [-1.7986,  3.4100,  0.4803, -1.9221, -1.5034,  4.7513, -3.4758, -3.9145],
        [-3.2551, -9.6171, -0.4902,  1.0551,  0.5027, -0.2759,  3.1721, -3.6231],
        [-3.3327,  5.2450,  1.2443, -1.9639, -4.3145,  4.5994, -3.4263,  0.1847],
        [-6.1774,  6.2514,  0.5321, -3.1332, -4.9116, -2.0588,  0.4785,  1.9875]])
0.9707003831863403, 0.9749341607093811, 13.919997215270996
torch.Size([4, 8, 8])
tensor([[ 0.2123,  1.3244, -0.5606, -0.1140, -0.0462,  1.6279, -0.3083,  1.7580],
        [ 0.2241,  0.0253,  0.7143,  0.2585, -0.4609, -0.9229,  0.4422, -0.6701],
        [ 0.6807,  0.6339, -0.0842,  0.2259, -0.2732, -0.7638,  0.2183,  0.4

In [ ]:
# why scaling is important
# first with low variance, softmax works fine
print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1))
# second with high variance, softmax skews towards max
print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1))

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])
tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])
